In [11]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd


'''
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Wed 16</h2>
<span data-testid="TemperatureValue" class="DetailsSummary--highTempValue--3x6cL">57°</span>
<span class="DailyContent--daypartDate--3MM0J">Wed 09</span>
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Thu 10</h2>
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Fri 11</h2>
<span data-testid="TemperatureValue" class="DailyContent--temp--_8DL5">56°</span>
<p data-testid="wxPhrase" class="DailyContent--narrative--3AcXd">Partly cloudy this evening with more clouds for overnight. Low 44F. Winds light and variable.</p>
'''


In [2]:
def fahrToCelsius(f):
    return round((f-32)*(5/9), 1)

def parseTemp(t):
    t = t.replace('°', '')
    highT = float(t[:t.find('/')])
    lowT = float(t[t.find('/')+1:])
    return f"{fahrToCelsius(highT)}°C/{fahrToCelsius(lowT)}°C"


In [13]:
page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")

soup = BeautifulSoup(page.content, 'html.parser')

tendays = soup.find("div", {"class" : "DailyForecast--DisclosureList--350ZO"})

weatherData = {}
dates = []
conditions = []
temperatures = []
itemNames = []

for i in range(1, 11):
    day = tendays.find("div", {"id" : f"detailIndex{i}"})

    dates.append(list(day.children)[0].get_text() + '.12')
    temperatures.append(parseTemp(list(day.children)[1].get_text()))
    itemNames.append(f"detailIndex{i}")

    rainChance = list(day.children)[3].get_text()
    windSpeed = list(day.children)[4].get_text()
    conditions.append(list(day.children)[2].get_text() + f', with {rainChance} chance of rain and {windSpeed} wind.')

weatherData['Temperature:'] = temperatures
weatherData['Conditions:'] = conditions
weatherData['Item Name:'] = itemNames

df = pd.DataFrame(weatherData, index=dates)
df

# :D 

,Temperature:,Conditions:,Item Name:
Thu 10.12,12.8°C/5.6°C,"Mostly Sunny, with 10% chance of rain and W 18...",detailIndex1
Fri 11.12,11.7°C/6.1°C,"Partly Cloudy, with 5% chance of rain and NW 9...",detailIndex2
Sat 12.12,12.8°C/7.2°C,"Showers, with 55% chance of rain and S 9 mph w...",detailIndex3
Sun 13.12,13.3°C/7.2°C,"Rain, with 65% chance of rain and SSW 12 mph w...",detailIndex4
Mon 14.12,13.3°C/5.6°C,"Partly Cloudy, with 15% chance of rain and NW ...",detailIndex5
Tue 15.12,12.8°C/6.1°C,"Partly Cloudy, with 25% chance of rain and W 8...",detailIndex6
Wed 16.12,12.8°C/6.7°C,"PM Showers, with 40% chance of rain and W 8 mp...",detailIndex7
Thu 17.12,13.3°C/6.1°C,"Showers, with 40% chance of rain and W 10 mph ...",detailIndex8
Fri 18.12,12.8°C/5.6°C,"AM Showers, with 35% chance of rain and NW 9 m...",detailIndex9
Sat 19.12,13.9°C/5.6°C,"AM Showers, with 35% chance of rain and NW 7 m...",detailIndex10
